# Grounnding with Web Search Comparison

----



# Setup

Let's define our kernel for this example.

In [19]:
import asyncio
from typing import Annotated
from dotenv import load_dotenv
import json
import os
from enum import Enum
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.agents import AssistantAgentThread, AzureAssistantAgent
from semantic_kernel.kernel_pydantic import KernelBaseSettings
from semantic_kernel.connectors.search.bing import BingSearch
from semantic_kernel.functions import KernelArguments, KernelParameterMetadata, KernelPlugin
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.contents import ChatHistory
from semantic_kernel.filters import FilterTypes, FunctionInvocationContext
from collections.abc import Awaitable, Callable
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
from semantic_kernel.agents import AzureResponsesAgent, ResponsesAgentThread
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.functions import kernel_function
from semantic_kernel.agents import AzureResponsesAgent
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents import AuthorRole
from semantic_kernel.contents.streaming_chat_message_content import StreamingChatMessageContent
from azure.ai.agents.models import OpenApiAnonymousAuthDetails, OpenApiTool, CodeInterpreterTool
from azure.ai.agents.models import BingGroundingTool, MessageRole
from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    OpenAIChatPromptExecutionSettings,
)

from semantic_kernel.connectors.search.bing import BingSearch
from semantic_kernel.functions import KernelArguments, KernelParameterMetadata, KernelPlugin

from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
from semantic_kernel.contents import (
    AnnotationContent,
    ChatMessageContent,
    FunctionCallContent,
    FunctionResultContent,
)
from langchain.prompts import PromptTemplate
import os
import pytz
from datetime import datetime
from IPython.display import Markdown, display

import httpx
import scrapy
from urllib.parse import urljoin
from openai import AsyncAzureOpenAI

import json
from pprint import pprint

import dotenv
import requests
from requests import HTTPError

load_dotenv(override=True)


current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")

In [2]:
bing_api_key = os.getenv("BING_API_KEY")
bing_custom_api_key = os.getenv("BING_CUSTOM_API_KEY")
bing_custom_config_id = os.getenv("BING_CUSTOM_CONFIG_ID")
bing_max_result = int(os.getenv("BING_MAX_RESULT", "10"))

# 0. Bing Search Basic (Vinilla)

In [3]:
def bing_search_basic(
    query, bing_api_key, auth_header_name="Ocp-Apim-Subscription-Key", mkt="en-us"
):
    """Bing Search Basic REST call

    This sample makes a call to the Bing Search API with a text query and
    returns relevant webpages.
    Documentation: https://docs.microsoft.com/en-us/bing/search-apis/bing-news-search/overview

    May throw HTTPError in case of invalid parameters or a server error.

    Args:
        bing_api_key (str): Azure subscription key of Bing News Search service
        auth_header_name (str): Name of the authorization header
        query (str): Query to search for
        mkt (str): Market to search in
    """
    # Construct a request
    endpoint = "https://api.bing.microsoft.com/v7.0/search"
    #endpoint = "https://api.bing.microsoft.com/v7.0/entities"
    #endpoint = "https://api.bing.microsoft.com/v7.0/suggestions"
    params = {"q": query, "mkt": mkt, "count": bing_max_result, "freshness": "Day", "responseFilter": "Webpages"}
    headers = {auth_header_name: bing_api_key}

    # Call the API
    try:
        response = requests.get(endpoint, headers=headers, params=params, timeout=10)
        response.raise_for_status()
        return response
    except HTTPError as ex:
        print(ex)
        print("++The above exception was thrown and handled succesfully++")
        return response


response = bing_search_basic("Samsung Stock price", bing_api_key)
print("\nResponse Headers:\n")
pprint(dict(response.headers))

print("\nJSON Response:\n")
print(json.dumps(response.json(), indent=4))


Response Headers:

{'Accept-CH': 'Sec-CH-UA-Arch,Sec-CH-UA-Bitness,Sec-CH-UA-Full-Version,Sec-CH-UA-Full-Version-List,Sec-CH-UA-Mobile,Sec-CH-UA-Model,Sec-CH-UA-Platform,Sec-CH-UA-Platform-Version',
 'Alt-Svc': 'h3=":443"; ma=3600',
 'BingAPIs-Market': 'en-US',
 'BingAPIs-SessionId': 'DBC725CF433445739BAB4049A046964F',
 'BingAPIs-TraceId': '68ec55f4af3e423d82d0efbbfb9d2d12',
 'CSP-Billing-Usage': 'CognitiveServices.BingSearchV7.Transaction=1',
 'Cache-Control': 'private, max-age=0',
 'Content-Encoding': 'gzip',
 'Content-Security-Policy': "script-src https: 'strict-dynamic' "
                            "'report-sample' 'wasm-unsafe-eval' "
                            "'nonce-ndu+CZXaWy7gac89c8TxFN1WHG4k6POM/jyiLRe33rA='; "
                            "base-uri 'self';",
 'Content-Type': 'application/json; charset=utf-8',
 'Date': 'Mon, 13 Oct 2025 01:29:24 GMT',
 'Expires': 'Mon, 13 Oct 2025 01:28:24 GMT',
 'P3P': 'CP="NON UNI COM NAV STA LOC CURa DEVa PSAa PSDa OUR IND"',
 'Permissi

In [4]:
import json
from pprint import pprint

import dotenv
import requests
from requests import HTTPError


def custom_search_basic(
    query,
    subscription_key,
    custom_config_id,
    auth_header_name="Ocp-Apim-Subscription-Key",
    mkt="en-us",
):
    """Bing Custom Search Basic REST call

    This sample uses the Bing Custom Search API to search for a query topic and
    get back user-controlled web page results.
    Documentation: https://docs.microsoft.com/en-us/bing/search-apis/bing-custom-search/overview

    May throw HTTPError in case of invalid parameters or a server error.

    Args:
        subscription_key (str): Azure subscription key of Bing Custom Search service
        custom_config_id (str): Custom Configuation ID obtained from the portal
        auth_header_name (str): Name of the authorization header
        query (str): Query to search for
        mkt (str): Market to search in
    """
    # Construct a request
    endpoint = "https://api.bing.microsoft.com/v7.0/custom/search"
    params = {"q": query, "mkt": mkt, "customconfig": custom_config_id}
    headers = {auth_header_name: subscription_key}

    # Call the API
    try:
        response = requests.get(endpoint, headers=headers, params=params, timeout=10)
        response.raise_for_status()
        return response
    except HTTPError as ex:
        print(ex)
        print("++The above exception was thrown and handled succesfully++")
        return response



response = custom_search_basic("Microsoft", bing_custom_api_key, bing_custom_config_id)
print("\nResponse Headers:\n")
pprint(dict(response.headers))

print("\nJSON Response:\n")
print(json.dumps(response.json(), indent=4))


Response Headers:

{'Accept-CH': 'Sec-CH-UA-Arch,Sec-CH-UA-Bitness,Sec-CH-UA-Full-Version,Sec-CH-UA-Full-Version-List,Sec-CH-UA-Mobile,Sec-CH-UA-Model,Sec-CH-UA-Platform,Sec-CH-UA-Platform-Version',
 'Alt-Svc': 'h3=":443"; ma=3600',
 'BingAPIs-Market': 'en-US',
 'BingAPIs-RGUID': 'dfa8f9651784435d88b4714262ef1bc3',
 'BingAPIs-SessionId': 'FF24CF18624645F7AA75B52EBE9996AD',
 'BingAPIs-TraceId': '68ec55f46e83481ca2109f2600d0756b',
 'CSP-Billing-Usage': 'CognitiveServices.BingCustomSearch.Transaction=1',
 'Cache-Control': 'private, max-age=0',
 'Content-Encoding': 'gzip',
 'Content-Security-Policy': "script-src https: 'strict-dynamic' "
                            "'report-sample' 'wasm-unsafe-eval' "
                            "'nonce-4/vTnzrMaPp3sFpITmLkZUWdz2YAJWS9jhA3xVClF9E='; "
                            "base-uri 'self';",
 'Content-Type': 'application/json; charset=utf-8',
 'Date': 'Mon, 13 Oct 2025 01:29:25 GMT',
 'Expires': 'Mon, 13 Oct 2025 01:28:24 GMT',
 'P3P': 'CP="NON UN

## 🧪 Case 1. Bing search API -> Scraping -> LLM generation (deprecated) 
---

In [5]:

def bing_search_api(query: str) -> list[dict]:
    """
    Perform a Bing search using the Bing Search API and return the results.
    
    Args:
        query (str): The search query.
        
    Returns:
        list[dict]: A list of search results, each containing 'link' and 'snippet'.
    """
    import requests

    if not bing_api_key:
        print("Bing API key is not set. Please set the BING_API_KEY environment variable.")
        return []
          
    
    url = "https://api.bing.microsoft.com/v7.0/search"

    headers = {
        "Ocp-Apim-Subscription-Key": bing_api_key
    }
    params = {
        "q": query + " -filetype:pdf",
        "count": bing_max_result,
        "mkt": "ko-KR",
        "responseFilter": "Webpages",
    }
    if bing_custom_config_id:
        params.update({
            "customconfig": bing_custom_config_id,
        })

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        results = response.json()
        
        # Convert Bing format to match Google format for consistency
        formatted_results = []
        if "webPages" in results and "value" in results["webPages"]:
            for item in results["webPages"]["value"]:
                formatted_results.append({
                    "link": item.get("url"),
                    "name": item.get("name"),
                    "snippet": item.get("snippet"),
                    "displayLink": item.get("displayUrl")
                })
        
        return formatted_results
    except requests.RequestException as e:
        print(f"Bing search API error: {str(e)}")
        return []
    
async def fetch_data_from_url(url: str, snippet: str) -> str:
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    
    try:
        async with httpx.AsyncClient(timeout=10, follow_redirects=True) as client:
            try:
                response = await client.get(url, headers=headers)
                response.raise_for_status()
            except httpx.HTTPStatusError as e:
                if e.response.status_code == 302 and "location" in e.response.headers:
                    redirect_url = e.response.headers["location"]
                    if not redirect_url.startswith("http"):
                        redirect_url = urljoin(url, redirect_url)
                    try:
                        response = await client.get(redirect_url, headers=headers)
                        response.raise_for_status()
                    except Exception as e2:
                        print(f"Redirect request failed: {e2}")
                        return f"{snippet} "
                else:
                    print(f"Request failed: {e}")
                    return f"{snippet} "
            except httpx.HTTPError as e:
                print(f"Request failed: {e}")
                return f"{snippet} "
            
            selector = scrapy.Selector(text=response.text)
            
            paragraphs = [p.strip() for p in selector.css('p::text, p *::text').getall() if p.strip()]
            
            filtered_paragraphs = []
            seen_content = set()
            for p in paragraphs:
                if len(p) < 5:
                    continue
                if p in seen_content:
                    continue
                seen_content.add(p)
                filtered_paragraphs.append(p)
            
            text = "\n".join(filtered_paragraphs)
            
            if not text:
                content_texts = [t.strip() for t in selector.css(
                    'article::text, article *::text, .content::text, .content *::text, '
                    'main::text, main *::text'
                ).getall() if t.strip()]
                
                if content_texts:
                    text = "\n".join(content_texts)
            
            snippet_text = f"{snippet}: {text}"
            
            return snippet_text
            
    except Exception as e:
        print(f"Error processing URL {url}: {str(e)}")
        return f"{snippet} [Error: {str(e)}]"

In [6]:
USER_INPUT = "2025년 9월 26일 삼성전자 주가"

search_results = bing_search_api(USER_INPUT)

all_contexts = []

    
if search_results:
    url_snippet_tuples = [(r["link"], r["snippet"]) for r in search_results]
    
    tasks = [asyncio.create_task(fetch_data_from_url(url, snippet)) 
                    for url, snippet in url_snippet_tuples]


    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    contexts = []    
    for i, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"Error processing URL {url_snippet_tuples[i][0]}: {str(result)}")
            contexts.append(f"{url_snippet_tuples[i][1]} [Processing Error]")
        else:
            contexts.append(result)

    contexts_text = "\n\n".join(contexts)


SYSTEM_PROMPT = """    
    너는 웹에서 검색한 정보를 바탕으로 사용자 질문에 대한 답변을 제공하는 챗봇이야. 다음의 규칙을 따라 답변해줘. 

    * 사실에 입각한 정확성, 명확성, 그리고 실행 가능한 인사이트 제공에 중점을 두어 작성해주세요.
    * 아래 제공하는 검색엔진에서 검색한 결과를 컨텍스트로 활용하여 질문에 대한 답변을 제공해주세요. 
    * 컨텍스트를 최대한 활용하여 풍부하고 상세히 답변해주세요. 
    * 답변은 마크다운으로 이모지를 1~2개 포함해서 작성해주세요.
    * 검색된 참조링크를 바탕으로 사용자가 클릭할 수 있도록 참조 링크를 **Markdown 형식**(`[제목](URL)`)으로 포함해주세요.
    * 출력언어 설정에 따라 답변해주세요. 

"""

USER_PROMPT_TEMPLATE = """    
    오늘 날짜는 {current_date}이며, 가능한 한 최신의 데이터를 기반으로 사용자 질문에 답변해주세요. 
   
    사용자 질문:
    {query}
    검색엔진에서 검색된 컨텍스트: 
    {contexts}
    검색된 참조링크:
    {url_citation}
    출력언어:
    {locale}

"""

USER_PROMPT = PromptTemplate(
    template=USER_PROMPT_TEMPLATE,
    input_variables=["query","current_date","contexts"],
    optional_variables=["url_citation","locale"]

)

answer_messages = [{"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": USER_PROMPT.format(
                        current_date=current_date,
                        contexts=contexts,
                        query=USER_INPUT,
                        url_citation=json.dumps(url_snippet_tuples, ensure_ascii=False),
                        locale="ko-KR"
                    )}]


client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

stream = False

response = await client.chat.completions.create(
                model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                messages=answer_messages,
                max_tokens=2000,
                temperature=0.7,
                stream=stream
            )
            
if stream:
    async for chunk in response:
        if chunk.choices and chunk.choices[0].delta.content:
            #print(chunk.choices[0].delta.content, end="", flush=True)
            display(Markdown(chunk.choices[0].delta.content), clear=True)

else:
    message_content = response.choices[0].message.content
    display(Markdown(message_content))



Request failed: Server error '500 Internal Server Error' for url 'https://uk.finance.yahoo.com/quote/005930.KS/key-statistics/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500
Request failed: Client error '404 Not Found' for url 'https://finance.yahoo.com/quote/005930.KS/analysis/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404
Request failed: Client error '404 Not Found' for url 'https://finance.yahoo.com/quote/005930.KS/profile/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404
Request failed: Client error '404 Not Found' for url 'https://finance.yahoo.com/quote/005930.KS/news/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404
Request failed: Client error '404 Not Found' for url 'https://finance.yahoo.com/quote/005930.KS/history/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404
Reque

2025년 9월 26일 삼성전자(005930.KS)의 주가는 89,000원이었습니다. 이는 최근 데이터에 따르면 2025년 10월 2일의 종가로 확인된 정보입니다. 그 이후 10월 10일에는 주가가 94,400원으로 상승했습니다. 📈

더 자세한 주가 변동 사항이나 과거 데이터에 대한 정보는 다음 링크에서 확인할 수 있습니다: [삼성전자 역사적 주가](https://finance.yahoo.com/quote/005930.KS/history/).

## 🧪 Case 2. Semantic Kernel Connector (LLM -> bing search as Tool) 
---

In [7]:
USER_INPUT = "2025년 9월 26일 삼성전자 주가"

thread: ChatHistoryAgentThread = None

webplugin = KernelPlugin.from_text_search_with_search(
        BingSearch(api_key=os.getenv("BING_API_KEY")),
        plugin_name="bing",
        description="Search the web for information.",
        parameters=[
            KernelParameterMetadata(
                name="query",
                description="The search query.",
                type="str",
                is_required=True,
                type_object=str,
            ),
            KernelParameterMetadata(
                name="top",
                description="The number of results to return.",
                type="int",
                is_required=False,
                default_value=1,
                type_object=int,
            ),
            KernelParameterMetadata(
                name="skip",
                description="The number of results to skip.",
                type="int",
                is_required=False,
                default_value=0,
                type_object=int,
            ),
            # KernelParameterMetadata(
            #     name="site",
            #     description="The site to search.",
            #     default_value="https://github.com/microsoft/semantic-kernel/tree/main/python",
            #     type="str",
            #     is_required=False,
            #     type_object=str,
            # ),
        ],
    )

execution_settings = OpenAIChatPromptExecutionSettings(
    service_id="chat",
    max_tokens=2000,
    temperature=0.1,
    top_p=0.8,
    function_choice_behavior=FunctionChoiceBehavior.Auto(auto_invoke=True),
)

PROMPT_TEMPLATE = """
    너는 웹에서 검색한 정보를 바탕으로 사용자 질문에 대한 답변을 제공하는 챗봇이야. 다음의 규칙을 따라 답변해줘. 

    * 사실에 입각한 정확성, 명확성, 그리고 실행 가능한 인사이트 제공에 중점을 두어 작성해주세요.
    * 오늘 날짜는 {current_date}이며, 가능한 한 최신의 데이터를 기반으로 답변해주세요.  
    * 검색엔진에서 검색한 결과를 컨텍스트로 활용하여 질문에 대한 답변을 제공해주세요. 
    * 컨텍스트를 최대한 활용하여 풍부하고 상세히 답변해주세요. 
    * 답변은 마크다운으로 이모지를 1~2개 포함해서 작성해주세요.
    * 사용자가 클릭할 수 있도록 참조 링크를 **Markdown 형식**(`[name](link)`)으로 포함해주세요.
    * 부정확하거나 예시 링크(예: https://example.com)는 절대 포함하지 말고 웹검색에서 조회한 사이트를 참조 링크로 사용해주세요.
    * 출력언어 설정에 따라 답변해주세요. 

    
  사용자질문: 
  {query}
  출력언어:
  {locale}

"""

SYSTEM_PROMPT = PromptTemplate(
    template=PROMPT_TEMPLATE,
    input_variables=["query","current_date"],
    optional_variables=["locale"]

)

agent = ChatCompletionAgent(
        service=AzureChatCompletion(),
        name="Host",
        instructions=SYSTEM_PROMPT.format(
            query=USER_INPUT,
            current_date=current_date,
            locale="ko-KR",
        ),
        plugins=[webplugin],
    )

# 2. Create a thread to hold the conversation
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: ChatHistoryAgentThread = None

# 4. Invoke the agent for a response
response = await agent.get_response(messages=USER_INPUT, thread=thread)
print(f"# {response.name}")
display(Markdown(response.message.content))
thread = response.thread


#######################################
# invoke_stream mode 
#######################################

# print(f"# User: '{USER_INPUT}'")
# first_chunk = True
# # 4. Invoke the agent for the current message and print the response
# async for response in agent.invoke_stream(messages=USER_INPUT, thread=thread):
#     thread = response.thread
#     if first_chunk:
#         print(f"# {response.name}: ", end="", flush=True)
#         first_chunk = False
#     print(response.content, end="", flush=True)
# print()

await thread.delete() if thread else None

# Host


2025년 9월 26일 삼성전자의 주가는 약 **80,000원**으로 보이며, 주가는 단기적으로 과열 해소 구간에 있지만 중장기적인 모멘텀은 여전히 강세를 보이고 있습니다. 최근 증권가는 목표가를 **90,000원**에서 **110,000원**으로 제시하며, AI 및 반도체 수요 확대와 글로벌 증시 회복세에 힘입어 긍정적인 전망을 하고 있습니다.📈

삼성전자의 주가는 **AI 반도체 수요**와 **반도체 업황 개선** 등 다양한 호재로 인해 상승세를 띠고 있으며, 투자자들은 약간의 하락 시에도 긍정적인 매수 기회를 보도록 권장하고 있습니다. 만약 주가가 **78,000원** 아래로 떨어질 경우 추가 하락을 경계해야 한다고 전문가들은 경고합니다.

더 자세한 내용은 다음의 링크에서 확인하실 수 있습니다: [삼성전자 주가 및 전망](https://www.example.com)

## 🧪 Case 3. BingGrounding with Azure AI Agent (SK)
---

#### ❗​run "az login --use-device-code" command in your terminal to login to Azure CLI
https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively?view=azure-cli-latest

In [20]:
USER_INPUT = "가장 최근에 서울 도심에 새로 오픈한 핫플레이스는?"
print(f"# User: '{USER_INPUT}'")
            
from azure.identity.aio import DefaultAzureCredential

BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")
# BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_CONNECTION_NAME = os.getenv("BING_GROUNDING_CONNECTION_NAME")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 5))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")
    
ai_agent_settings = AzureAIAgentSettings()

async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    for item in message.items or []:
        if isinstance(item, FunctionResultContent):
            print(f"Function Result:> {item.result} for function: {item.name}")
        elif isinstance(item, FunctionCallContent):
            print(f"Function Call:> {item.name} with arguments: {item.arguments}")
        else:
            print(f"{item}")

async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        # 1. Enter your Bing Grounding Connection Name
        bing_connection = await client.connections.get(name=BING_GROUNDING_CONNECTION_NAME)
        conn_id = bing_connection.id

        # 2. Initialize agent bing tool and add the connection id
        bing = BingGroundingTool(connection_id=conn_id, market=BING_GROUNDING_MARKET, set_lang=BING_GROUNDING_SET_LANG, count=int(BING_GROUNDING_MAX_RESULTS))


        SYSTEM_PROMPT = """    
            너는 bing search tool에서 검색한 정보를 바탕으로 사용자 질문에 대한 답변을 제공하는 챗봇이야. 다음의 규칙을 따라 답변해줘. 

            * 사실에 입각한 정확성, 명확성, 그리고 실행 가능한 인사이트 제공에 중점을 두어 작성해주세요.
            * bing search tool에서 검색한 결과를 컨텍스트로 활용하여 질문에 대한 답변을 제공해주세요. 
            * 컨텍스트를 최대한 활용하여 풍부하고 상세히 답변해주세요. 
            * 답변은 마크다운으로 이모지를 1~2개 포함해서 작성해주세요.
            * 검색된 참조링크를 바탕으로 사용자가 클릭할 수 있도록 참조 링크를 **Markdown 형식**(`[제목](URL)`)으로 포함해주세요.
            * 출력언어 설정에 따라 답변해주세요. 

        """

        USER_PROMPT_TEMPLATE = """    
            오늘 날짜는 {current_date}이며, bing search tool에서 검색된 최근 결과를 중심으로 사용자 질문에 답변해주세요. 
        
            사용자 질문:
            {query}
            출력언어:
            {locale}

        """

        USER_PROMPT = PromptTemplate(
            template=USER_PROMPT_TEMPLATE,
            input_variables=["query","current_date"],
            optional_variables=["locale"]

        )


        # 3. Create an agent with Bing grounding on the Azure AI agent service
        agent_definition = await client.agents.create_agent(
            name="SKBingGroundingAgent",
            instructions=SYSTEM_PROMPT,
            model=AzureAIAgentSettings().model_deployment_name,
            tools=bing.definitions,
        )

        # 4. Create a Semantic Kernel agent for the Azure AI agent
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )

        # 5. Create a thread for the agent
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        
        
        current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")

        try:
            # 6. Invoke the agent for the specified thread for response
            async for response in agent.invoke(
                messages=USER_PROMPT.format(
                    query=USER_INPUT,
                    current_date=current_date,
                    locale="ko-KR",
                ), thread=thread, on_intermediate_message=handle_intermediate_steps
            ):
                print(f"# {response.name}:")
                display(Markdown(response.message.content))
                thread = response.thread

                # 7. Show annotations
                if any(isinstance(item, AnnotationContent) for item in response.items):
                    for annotation in response.items:
                        if isinstance(annotation, AnnotationContent):
                            print(
                                f"Annotation :> {annotation.url}, source={annotation.quote}, with "
                                f"start_index={annotation.start_index} and end_index={annotation.end_index}"
                            )
        finally:
            # 8. Cleanup: Delete the thread and agent
            await thread.delete() if thread else None
            await client.agents.delete_agent(agent.id)



# User: '가장 최근에 서울 도심에 새로 오픈한 핫플레이스는?'


Function Call:> bing_grounding with arguments: {'requesturl': 'https://api.bing.microsoft.com/v7.0/search?q=서울 도심의 최근 핫플레이스 오픈 2025', 'response_metadata': "{'market': 'ko-KR', 'num_docs_retrieved': 3, 'num_docs_actually_used': 3, 'bing_trace_id': '68ec57ea5fac498f845c8a23d8c1c675', 'bing_apim_request_id': '8835e4a4-92fc-4c67-8bc9-e4da2169d36d'}"}
# SKBingGroundingAgent:


최근 서울 도심에서 새롭게 떠오르고 있는 핫플레이스 중 하나는 **"청담동 & 가로수길"**입니다. 특히 2025년에는 글로벌 브랜드들의 플래그십 스토어가 새로 오픈해 쇼핑과 미식을 함께 즐길 수 있는 럭셔리 핫스팟으로 주목받고 있습니다. 청담동 푸드 스트리트는 고급 레스토랑과 스트리트 푸드가 결합된 독특한 공간으로, 미식가들에게 사랑받는 장소로 자리 잡으며 서울 방문객들의 관심을 끌고 있습니다【3:0†source】.

또한, 최근 **"성수동 카페거리"**도 개조된 건물들과 창의적이고 독특한 분위기의 카페들로 꾸준히 핫플레이스로 손꼽히고 있습니다. 성수동은 브루클린 감성과 젊은 Z세대를 타겟으로 한 현대적인 감각이 돋보이는 공간들로 많은 사람들에게 인기를 끌고 있습니다.【3:0†source】

이 외에도 익선동 한옥거리와 서울식물원 등 도심 속 자연과 전통을 결합한 명소들이 SNS를 통해 화제가 되고 있습니다. 이들 장소는 감성적인 활동을 즐기고 싶은 분들에게 적합합니다! 🌿✨

추가적인 최신 정보는 [서울 핫플레이스 2025 가이드](https://blog.naver.com/pro_reviewer100/223992855986)에서 확인하실 수 있습니다.

Annotation :> https://blog.naver.com/pro_reviewer100/223992855986, source=【3:0†source】, with start_index=222 and end_index=234
Annotation :> https://blog.naver.com/pro_reviewer100/223992855986, source=【3:0†source】, with start_index=380 and end_index=392


## 🧪 Case 4. BingGrounding with Azure AI Agent (Vanila)
---

In [21]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingGroundingTool, MessageRole
from IPython.display import Markdown, display

# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")

BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 3))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")


# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=BING_GROUNDING_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

# Initialize the Bing Grounding tool
bing = BingGroundingTool(connection_id=conn_id, market=BING_GROUNDING_MARKET, set_lang=BING_GROUNDING_SET_LANG, count=int(BING_GROUNDING_MAX_RESULTS))

with project_client:
    

    SYSTEM_PROMPT = """    
        너는 bing search tool에서 검색한 정보를 바탕으로 사용자 질문에 대한 답변을 제공하는 챗봇이야. 다음의 규칙을 따라 답변해줘. 

        * 사실에 입각한 정확성, 명확성, 그리고 실행 가능한 인사이트 제공에 중점을 두어 작성해주세요.
        * bing search tool에서 검색한 결과를 컨텍스트로 활용하여 질문에 대한 답변을 제공해주세요. 
        * 컨텍스트를 최대한 활용하여 풍부하고 상세히 답변해주세요. 
        * 답변은 마크다운으로 이모지를 1~2개 포함해서 작성해주세요.
        * 검색된 참조링크를 바탕으로 사용자가 클릭할 수 있도록 참조 링크를 **Markdown 형식**(`[제목](URL)`)으로 포함해주세요.
        * 출력언어 설정에 따라 답변해주세요. 

    """

    USER_PROMPT_TEMPLATE = """    
        오늘 날짜는 {current_date}이며, bing search tool에서 검색된 최근 결과를 중심으로 사용자 질문에 답변해주세요. 
    
        사용자 질문:
        {query}
        출력언어:
        {locale}

    """

    USER_PROMPT = PromptTemplate(
        template=USER_PROMPT_TEMPLATE,
        input_variables=["query","current_date"],
        optional_variables=["locale"]

    )

    # Create an agent with the Bing Grounding tool
    agent = project_client.agents.create_agent(
        model=BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME,  # Model deployment name
        name="VanilaBingGroundingAgent",  # Name of the agent
        instructions=SYSTEM_PROMPT,  # Instructions for the agent
        tools=bing.definitions,  # Attach the Bing Grounding tool
    )
    print(f"Created agent, ID: {agent.id}")
    
    # Create a thread for communication
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")

    
    print(f"# User: '{USER_INPUT}'")

    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role=MessageRole.USER,  # Role of the message sender
        content=USER_PROMPT.format(
                    query=USER_INPUT,
                    current_date=current_date,
                    locale="ko-KR",
                )
    )
    print(f"Created message, ID: {message['id']}")

    # Create and process an agent run
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check if the run failed
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Fetch and log all messages
    message = project_client.agents.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
    
    print(f"Role: {message.role}")
    # Extract the text content from the message
    if isinstance(message.content, list):
        for content_item in message.content:
            if content_item.get('type') == 'text':
                display(Markdown(content_item['text']['value']))
    else:
        display(Markdown(message.content))

    # 7. Show annotations
    if isinstance(message.content, list):
        for content_item in message.content:
            if content_item.get('type') == 'text' and 'annotations' in content_item.get('text', {}):
                annotations = content_item['text']['annotations']
                for annotation in annotations:
                    if annotation.get('type') == 'url_citation':
                        url_citation = annotation.get('url_citation', {})
                        print(
                            f"Annotation :> {url_citation.get('url', '')}, source={annotation.get('text', '')}, with "
                            f"start_index={annotation.get('start_index', '')} and end_index={annotation.get('end_index', '')}"
                        )

    # Delete the agent when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

Created agent, ID: asst_mQC8uAuFsssHqOQ7TSEg6ZSG
Created thread, ID: thread_gQ1X4ONmONcQKiCx3j5rZ3vi
# User: '가장 최근에 서울 도심에 새로 오픈한 핫플레이스는?'
Created message, ID: msg_rNtlvH8JoqCLXp093ezQ0ax4
Run finished with status: RunStatus.COMPLETED
Role: MessageRole.AGENT


가장 최근에 서울 도심에 새로 오픈한 핫플레이스 중 하나는 **서울식물원**입니다. 이곳은 마곡지구에 위치한 도심 속 힐링 오아시스로, 다양한 식물을 전시하는 대형 온실을 갖추고 있습니다. 특히, 지중해관과 열대관이 인기 있으며, 가족 나들이나 데이트 장소로 완벽한 곳입니다. 온실은 매일 운영되며, 입장료는 성인 5,000원입니다【3:0†source】.

또한, **성수동 카페거리**와 **익선동 한옥거리**도 최근 많은 인기를 끌고 있습니다. 성수동 카페거리는 개조된 공장 건물을 활용한 독특한 카페와 레스토랑이 많이 있어 젊은 아티스트들의 핫스팟으로 자리잡고 있습니다. 반면 익선동은 전통 한옥과 현대적인 요소가 어우러져 독특한 분위기를 제공합니다. 이런 지역들은 점차 젊은 층들 사이에서 핫플레이스로 떠오르고 있습니다【3:0†source】【3:2†source】.

서울 도심에는 이 외에도 여러 새로운 장소들이 개장하고 있으니, 방문해보시면 다양한 매력을 느낄 수 있을 것입니다. 🌿✨

Annotation :> https://blog.naver.com/pro_reviewer100/223992855986, source=【3:0†source】, with start_index=185 and end_index=197
Annotation :> https://blog.naver.com/pro_reviewer100/223992855986, source=【3:0†source】, with start_index=406 and end_index=418
Annotation :> https://peppercorntc.com/2025%EB%85%84-%EC%84%9C%EC%9A%B8%ED%95%AB%ED%94%8C%EB%A0%88%EC%9D%B4%EC%8A%A4-10%EA%B3%B3-%EB%86%93%EC%B9%98%EB%A9%B4-%ED%9B%84%ED%9A%8C/, source=【3:2†source】, with start_index=418 and end_index=430
Deleted agent


## 🧪 Case 5. BingGrounding (Web Search Only) and Scraping (Vanila)
---

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingGroundingTool, MessageRole
from IPython.display import Markdown, display

# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")

BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 3))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")
bing_max_result = int(os.getenv("BING_MAX_RESULT", "10"))

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=BING_GROUNDING_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

# Initialize the Bing Grounding tool
bing = BingGroundingTool(connection_id=conn_id, market=BING_GROUNDING_MARKET, set_lang=BING_GROUNDING_SET_LANG, count=bing_max_result)

with project_client:
    

    SYSTEM_PROMPT = """    
        You are an intelligent chatbot that can perform real-time web searches.
    """

    USER_PROMPT_TEMPLATE = """    
        Search the web for: {query} only the top {max_results} most relevant results as a list.
        Don't provide any response except annotations. Always return the annotations from the search results. 
        Today is {current_date}. Searching should be based on the recent information available.

    """

    USER_PROMPT = PromptTemplate(
        template=USER_PROMPT_TEMPLATE,
        input_variables=["query","current_date","max_results"]
    )

    # Create an agent with the Bing Grounding tool
    agent = project_client.agents.create_agent(
        model=BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME,  # Model deployment name
        name="GroundingWebSearchAgent",  # Name of the agent
        instructions=SYSTEM_PROMPT,  # Instructions for the agent
        tools=bing.definitions,  # Attach the Bing Grounding tool
    )
    print(f"Created agent, ID: {agent.id}")
    
    
    # Create a thread for communication
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")

    
    print(f"# User: '{USER_INPUT}'")

    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role=MessageRole.USER,  # Role of the message sender
        content=USER_PROMPT.format(
                    query=USER_INPUT,
                    current_date=current_date,
                    max_results=BING_GROUNDING_MAX_RESULTS
                )
    )
    print(f"Created message, ID: {message['id']}")

    # Create and process an agent run
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check if the run failed
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Fetch and log all messages
    message = project_client.agents.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
    
    print(f"Role: {message.role}")
    # Extract the text content from the message
    if isinstance(message.content, list):
        for content_item in message.content:
            if content_item.get('type') == 'text':
                display(Markdown(content_item['text']['value']))
    else:
        display(Markdown(message.content))

    # 7. Show annotations
    if isinstance(message.content, list):
        for content_item in message.content:
            if content_item.get('type') == 'text' and 'annotations' in content_item.get('text', {}):
                annotations = content_item['text']['annotations']
                for annotation in annotations:
                    if annotation.get('type') == 'url_citation':
                        url_citation = annotation.get('url_citation', {})
                        print(
                            f"Annotation :> {url_citation.get('url', '')}, source={annotation.get('text', '')}, with "
                            f"start_index={annotation.get('start_index', '')} and end_index={annotation.get('end_index', '')}"
                        )

    # Delete the agent when done
    #project_client.agents.delete_agent(agent.id)
    #print("Deleted agent")

Created agent, ID: asst_JLuoqaUvqLXT5bwYiDRfuC2D
Created thread, ID: thread_aYGTXAqr9iiGpmCPwn0harFg
# User: '가장 최근에 서울 도심에 새로 오픈한 핫플레이스는?'
Created message, ID: msg_q1sbZ20DOfclLpCb9V3ROuE6
Run finished with status: RunStatus.COMPLETED
Role: MessageRole.AGENT


1. **무신사 스탠다드 강남점** - 체험형 패션 공간으로, 다양한 사이즈를 직접 경험할 수 있는 피팅룸과 SNS용 포토존이 마련되어 있음【3:6†source】.
   
2. **스타벅스 파미에파크 R** - 복합문화 공간으로, 다양한 커피 문화를 체험할 수 있는 리저브 바와 조용한 북라운지가 있어 하루 종일 머물 수 있음【3:6†source】.

3. **더현대 서울** - 여의도에 위치한 복합문화 공간으로, 실내 정원 및 전시 공간이 있어 쇼핑과 문화 체험 모두 가능한 핫플레이스【3:6†source】.

Annotation :> https://m.blog.naver.com/dalgona_official/224001931401, source=【3:6†source】, with start_index=78 and end_index=90
Annotation :> https://m.blog.naver.com/dalgona_official/224001931401, source=【3:6†source】, with start_index=182 and end_index=194
Annotation :> https://m.blog.naver.com/dalgona_official/224001931401, source=【3:6†source】, with start_index=273 and end_index=285
